In [ ]:
#| default_exp read_data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import operator
from fastcore.test import *
from sys import platform
import numpy as np
from pathlib import Path
from typing import Tuple

from cupid_matching.matching_utils import Matching, _compute_margins, _get_singles

In [ ]:
#| export

def get_root_dir(
                    # no argument
    ) -> Path:      # the package directory
    """ returns the package directory """
    root_dir = Path.cwd().parent
#     if platform in ["linux", "linux2"]:        # we are deploying
#         root_dir = Path.cwd().parent
    return root_dir

In [ ]:
#| export

def read_margins(
    data_dir: Path      # the data directory
    ) -> Tuple[np.ndarray, np.ndarray]: 
    """reads and returns the margins for men and for women """
    nx = np.loadtxt(data_dir / "nx70n.txt")
    my = np.loadtxt(data_dir / "my70n.txt")
    return nx, my

def read_marriages(
    data_dir: Path      # the data directory
    ) -> Tuple[np.ndarray, np.ndarray]: 
    """ reads and returns the marriages and the variances"""
    muxy = np.loadtxt(data_dir / "muxy70nN.txt")
    varmus = np.loadtxt(data_dir / "varmus70nN.txt")
    return muxy, varmus

In [ ]:
#| export 

def reshape_varcov(
    varmus: np.ndarray,      #  muxy row major, then  mux0, then mu0y packed in both dimensions
    mus: Matching,           #  the original Matching
    n_households: int        #  the number of households we want
) -> tuple[np.ndarray]:      #  the 6 constituent blocks of the normalized variance-covariance
    """ splits the variance-covariance matrix 
    and renomalizes for a requested total number of households
    """
    muxy, mux0, mu0y, *_ = mus.unpack()
    ncat_men, ncat_women = muxy.shape
    n_prod_categories = ncat_men * ncat_women
    # first we reshape
    varmus_xyzt = varmus[:n_prod_categories, :n_prod_categories]
    varmus_xyz0 = varmus[:n_prod_categories,
                                n_prod_categories:(n_prod_categories + ncat_men)]
    varmus_xy0t = varmus[:n_prod_categories,
                                (n_prod_categories + ncat_men):]
    varmus_x0z0 = varmus[n_prod_categories:(n_prod_categories + ncat_men),
                                n_prod_categories:(n_prod_categories + ncat_men)]
    varmus_x00y = varmus[n_prod_categories:(n_prod_categories + ncat_men), 
                         (n_prod_categories + ncat_men):]
    varmus_0y0t = varmus[(n_prod_categories + ncat_men):, 
                         (n_prod_categories + ncat_men):]
    varcovs = (varmus_xyzt, varmus_xyz0, varmus_xy0t,
              varmus_x0z0, varmus_x00y, varmus_0y0t)
    # then we rescale
    n_households_mus = np.sum(muxy) + np.sum(mux0) + np.sum(mu0y)
    rescale_factor = n_households/n_households_mus
    rescale_factor2 =  rescale_factor*rescale_factor
    varcovs = tuple(v*rescale_factor2 for v in varcovs)
    return varcovs

In [ ]:
#| hide
muxy = np.array([[0, 2], [1,3], [4,5]])
nx = np.array([3, 5, 10])
my = np.array([7, 14])
n_types_men = nx.size
n_types_women = my.size
n_prod_categories = n_types_men*n_types_women
mus = Matching(muxy, nx, my)
mux0, mu0y = _get_singles(muxy, nx, my)
n_households = np.sum(muxy) + np.sum(mux0) + np.sum(mu0y)
new_nh = 331
varmus = np.random.uniform(size=(11, 11))
varcovs_rescaled = reshape_varcov(varmus, mus, new_nh)
varcovs_xyzt, varcovs_xyz0, varcovs_xy0t, \
              varcovs_x0z0, varcovs_x00y, varcovs_0y0t = varcovs_rescaled
i, j, k, l = (2, 1, 1, 1)
test_close(varcovs_xyzt[i*n_types_women+j,k*n_types_women+l], 
           varmus[i*n_types_women+j,k*n_types_women+l]*((new_nh/n_households)**2))
test_close(varcovs_x00y[i, j], 
           varmus[n_prod_categories+i, n_prod_categories+n_types_men+j]*((new_nh/n_households)**2))

In [ ]:
#| export 

def rescale_mus(
    mus: Matching,         # muxy, mux0, mu0y
    n_households: int | float    # the number of households we want
) -> Matching:  # the normalized Matching after rescaling
    """ normalizes the marriages and margins to a requested total number of households"""
    muxy, mux0, mu0y, nx, my = mus.unpack()
    n_households_mus = np.sum(muxy) + np.sum(mux0) + np.sum(mu0y)
    rescale_factor = n_households/n_households_mus
    muxy_norm = muxy * rescale_factor
    nx_norm = nx * rescale_factor
    my_norm = my  * rescale_factor
    mus_norm = Matching(muxy_norm, nx_norm, my_norm)
    return mus_norm

  

In [ ]:
#| hide
muxy = np.array([[0, 2], [1,3], [4,5]])
nx = np.array([3, 5, 10])
my = np.array([7, 14])
mus = Matching(muxy, nx, my)
mux0, mu0y = _get_singles(muxy, nx, my)
n_households = np.sum(muxy) + np.sum(mux0) + np.sum(mu0y)
new_nh = 331
mus_rescaled = rescale_mus(mus, new_nh)
muxy_rescaled, mux0_rescaled, mu0y_rescaled, nx_rescaled, my_rescaled = mus_rescaled.unpack()
test_close(muxy_rescaled[2, 1], muxy[2,1]*new_nh/n_households)
test_close(nx_rescaled[1], nx[1]*new_nh/n_households)
test_close(mu0y_rescaled[1], mu0y[1]*new_nh/n_households)


In [ ]:
#| export 

def _get_zeros_mu(
    mu: np.ndarray,
    eps: float=1e-9
) -> Tuple[bool, np.ndarray, float]:
    mu_size = mu.size
    nonzero_mu = mu[mu > eps]
    min_nonzero = np.min(nonzero_mu)
    n_zeros_mu = mu_size - nonzero_mu.size
    mu_has_zeros = (n_zeros_mu > 0)
    return mu_has_zeros, mu_size, min_nonzero
    
    
def remove_zero_cells(
    mus: Matching,         # muxy, mux0, mu0y, n, m
    coeff: int = 100 # default scale factor for delta
) -> Matching:  # the transformed muxy, mux0, mu0y, nx, my
    """add small number `delta` to 0-cells to avoid numerical issues"""
    muxy, mux0, mu0y, *_ = mus.unpack()
    zeros_muxy, muxy_size, min_muxy = _get_zeros_mu(muxy)
    zeros_mux0, mux0_size, min_mux0 = _get_zeros_mu(mux0)
    zeros_mu0y, mu0y_size, min_mu0y = _get_zeros_mu(mu0y)
    some_zeros = (zeros_muxy or zeros_mux0 or zeros_mu0y)
    if not some_zeros:
        return mus
    else:
        delta = min(min_muxy, min_mux0, min_mu0y)/coeff
        muxy_fixed = muxy.astype(float)
        mux0_fixed = mux0.astype(float)
        mu0y_fixed = mu0y.astype(float)
        n_cells = 0
        if zeros_muxy:
            muxy_fixed += delta
            n_cells += muxy_size
        if zeros_mux0:
            mux0_fixed += delta
            n_cells += mux0_size
        if zeros_mu0y:
            mu0y_fixed += delta
            n_cells += mu0y_size
        n_households = np.sum(muxy)+np.sum(mux0)+np.sum(mu0y)
        scale_factor = n_households/(n_households+delta*n_cells)
        muxy_fixed *= scale_factor
        mux0_fixed *= scale_factor
        mux0_fixed *= scale_factor
        nx_fixed, my_fixed = _compute_margins(muxy_fixed, mux0_fixed, mu0y_fixed)
        mus_fixed = Matching(muxy_fixed, nx_fixed, my_fixed)
        return mus_fixed

In [ ]:
#| hide
muxy = np.array([[0, 2], [1,3], [4,5]])
nx = np.array([3, 5, 10])
my = np.array([7, 14])
mus = Matching(muxy, nx, my)
mux0, mu0y = _get_singles(muxy, nx, my)
changed_mus = remove_zero_cells(mus)
changed_muxy, changed_mux0, changed_mu0y, changed_nx, changed_my = changed_mus.unpack()
for changed_mu, old_mu in zip([changed_muxy, changed_mux0, changed_mu0y, changed_nx, changed_my],
                       [muxy, mux0, mu0y, nx, my]):
    diff_mu = np.abs(changed_mu - old_mu)
    test(np.max(diff_mu), 0.1, operator.lt)
    test(np.min(changed_mu), 0.0, operator.gt)







In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()